In [18]:
from nba_api.stats.static import teams

nba_teams = teams.get_teams()
#Select the dictionary for the Celtics, which contains their Team ID
celtics = [team for team in nba_teams if team ['abbreviation'] == 'BOS'][0]
celtics_id = celtics['id']

In [19]:
from nba_api.stats.endpoints import leaguegamefinder

# Query for games where the Celtics were playing
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=celtics_id)
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()[0]
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42023,1610612738,BOS,Boston Celtics,0042300304,2024-05-27,BOS @ IND,W,240,105,40,89,0.449,14,44,0.318,11,14,0.786,12,32,44,22,10,6,11,18,3.0
1,42023,1610612738,BOS,Boston Celtics,0042300303,2024-05-25,BOS @ IND,W,239,114,39,83,0.470,16,46,0.348,20,26,0.769,7,29,36,24,7,12,7,18,3.0
2,42023,1610612738,BOS,Boston Celtics,0042300302,2024-05-23,BOS vs. IND,W,239,126,47,88,0.534,15,37,0.405,17,20,0.850,13,27,40,28,9,2,10,16,16.0
3,42023,1610612738,BOS,Boston Celtics,0042300301,2024-05-21,BOS vs. IND,W,265,133,47,99,0.475,15,45,0.333,24,30,0.800,13,30,43,30,11,5,14,15,5.0
4,42023,1610612738,BOS,Boston Celtics,0042300205,2024-05-15,BOS vs. CLE,W,239,113,42,80,0.525,19,43,0.442,10,12,0.833,11,32,43,32,6,6,12,15,15.0


In [20]:
games.groupby(games.SEASON_ID.str[-4:])[['GAME_ID']].count().loc['2015':]

,GAME_ID
SEASON_ID,
2015,103
2016,115
2017,113
2018,101
2019,102
2020,80
2021,115
2022,111
2023,106


In [21]:
# Subset the games to when the last 4 digits of SEASON_ID were 2017
games_2324 = games[games.SEASON_ID.str[-4:] == '2023']
games_2324.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42023,1610612738,BOS,Boston Celtics,0042300304,2024-05-27,BOS @ IND,W,240,105,40,89,0.449,14,44,0.318,11,14,0.786,12,32,44,22,10,6,11,18,3.0
1,42023,1610612738,BOS,Boston Celtics,0042300303,2024-05-25,BOS @ IND,W,239,114,39,83,0.470,16,46,0.348,20,26,0.769,7,29,36,24,7,12,7,18,3.0
2,42023,1610612738,BOS,Boston Celtics,0042300302,2024-05-23,BOS vs. IND,W,239,126,47,88,0.534,15,37,0.405,17,20,0.850,13,27,40,28,9,2,10,16,16.0
3,42023,1610612738,BOS,Boston Celtics,0042300301,2024-05-21,BOS vs. IND,W,265,133,47,99,0.475,15,45,0.333,24,30,0.800,13,30,43,30,11,5,14,15,5.0
4,42023,1610612738,BOS,Boston Celtics,0042300205,2024-05-15,BOS vs. CLE,W,239,113,42,80,0.525,19,43,0.442,10,12,0.833,11,32,43,32,6,6,12,15,15.0


In [22]:
raps_games_2324 = games_2324[games_2324.MATCHUP.str.contains('TOR')]
raps_games_2324.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
56,22023,1610612738,BOS,Boston Celtics,0022300563,2024-01-15,BOS @ TOR,W,240,105,33,82,0.402,16,39,0.410,23,25,0.920,5,48,53,20,3,7,12,16,9.0
65,22023,1610612738,BOS,Boston Celtics,0022300432,2023-12-29,BOS vs. TOR,W,239,120,43,88,0.489,17,39,0.436,17,20,0.850,13,39,52,29,3,6,16,17,2.0
84,22023,1610612738,BOS,Boston Celtics,0022300031,2023-11-17,BOS @ TOR,W,240,108,42,92,0.457,16,46,0.348,8,9,0.889,8,36,44,29,5,4,10,13,3.0
87,22023,1610612738,BOS,Boston Celtics,0022300174,2023-11-11,BOS vs. TOR,W,242,117,47,86,0.547,15,44,0.341,8,13,0.615,5,41,46,30,1,4,12,14,23.0


In [23]:
last_raps_game = raps_games_2324.sort_values('GAME_DATE').iloc[-1]
last_raps_game

SEASON_ID                     22023
TEAM_ID                  1610612738
TEAM_ABBREVIATION               BOS
TEAM_NAME            Boston Celtics
GAME_ID                  0022300563
GAME_DATE                2024-01-15
MATCHUP                   BOS @ TOR
WL                                W
MIN                             240
PTS                             105
FGM                              33
FGA                              82
FG_PCT                        0.402
FG3M                             16
FG3A                             39
FG3_PCT                        0.41
FTM                              23
FTA                              25
FT_PCT                         0.92
OREB                              5
DREB                             48
REB                              53
AST                              20
STL                               3
BLK                               7
TOV                              12
PF                               16
PLUS_MINUS                  

In [24]:
game_id = last_raps_game.GAME_ID
game_id

'0022300563'

In [25]:

# Get **all** the games so we can filter to an individual GAME_ID
result = leaguegamefinder.LeagueGameFinder()
all_games = result.get_data_frames()[0]
# Find the game_id we want
full_game = all_games[all_games.GAME_ID == game_id]
full_game

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
4798,22023,1610612738,BOS,Boston Celtics,0022300563,2024-01-15,BOS @ TOR,W,240,105,33,82,0.402,16,39,0.410,23,25,0.920,5,48,53,20,3,7,12,16,9.0
4819,22023,1610612761,TOR,Toronto Raptors,0022300563,2024-01-15,TOR vs. BOS,L,241,96,38,95,0.400,4,32,0.125,16,21,0.762,9,43,52,25,6,2,7,21,-9.0


In [27]:
import pandas as pd

def combine_team_games(df, keep_method='home'):
  #Join every row to all others with the same game ID.
  joined = pd.merge(df, df, suffixes=['_A', '_B'],
                    on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
  # Filter out any row that is joined to self.
  result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
  # Take action based on the keep_method flag.
  if keep_method is None:
    # return all the rows.
    pass
  if keep_method.lower() == 'home':
    # Keep rows where TEAM_A is the home team.
    result = result[result.MATCHUP_A.str.contains(' vs. ')]
  elif keep_method.lower() == 'away':
    # Keep rows where TEAM_A is the away team
    result = result[result.MATCHUO_A.str.contains(' @ ')]
  elif keep_method.lower() == 'winner':
    # Keep rows where TEAM_A won
    result = result[result.WL_A == 'W']
  elif keep_method.lower() == 'loser':
    # Keep rows where TEAM_A lost
    result = result[result.WL_A == 'L']
  else:
    raise ValueError(f'Invalid keep_method: {keep_method}')
  return result

# Combine the game rows into one. By default, the home team will be TEAM_A
game_df = combine_team_games(full_game)
game_df


,SEASON_ID,TEAM_ID_A,TEAM_ABBREVIATION_A,TEAM_NAME_A,GAME_ID,GAME_DATE,MATCHUP_A,WL_A,MIN_A,PTS_A,FGM_A,FGA_A,FG_PCT_A,FG3M_A,FG3A_A,FG3_PCT_A,FTM_A,FTA_A,FT_PCT_A,OREB_A,DREB_A,REB_A,AST_A,STL_A,BLK_A,TOV_A,PF_A,PLUS_MINUS_A,TEAM_ID_B,TEAM_ABBREVIATION_B,TEAM_NAME_B,MATCHUP_B,WL_B,MIN_B,PTS_B,FGM_B,FGA_B,FG_PCT_B,FG3M_B,FG3A_B,FG3_PCT_B,FTM_B,FTA_B,FT_PCT_B,OREB_B,DREB_B,REB_B,AST_B,STL_B,BLK_B,TOV_B,PF_B,PLUS_MINUS_B
2,22023,1610612761,TOR,Toronto Raptors,0022300563,2024-01-15,TOR vs. BOS,L,241,96,38,95,0.4,4,32,0.125,16,21,0.762,9,43,52,25,6,2,7,21,-9.0,1610612738,BOS,Boston Celtics,BOS @ TOR,W,240,105,33,82,0.402,16,39,0.41,23,25,0.92,5,48,53,20,3,7,12,16,9.0


In [28]:
from nba_api.stats.endpoints import playbyplayv2
pbp = playbyplayv2.PlayByPlayV2(game_id)
pbp = pbp.get_data_frames()[0]
pbp.head()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ID,PLAYER1_TEAM_CITY,PLAYER1_TEAM_NICKNAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
0,0022300563,2,12,0,1,7:41 PM,12:00,None,Start of 1st Period (7:41 PM EST),None,None,None,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,0
1,0022300563,4,10,0,1,7:41 PM,12:00,Jump Ball Barnes vs. Porzingis: Tip to White,None,None,None,None,4,1630567,Scottie Barnes,1.610613e+09,Toronto,Raptors,TOR,5,204001,Kristaps Porzingis,1.610613e+09,Boston,Celtics,BOS,5,1628401,Derrick White,1.610613e+09,Boston,Celtics,BOS,1
2,0022300563,7,2,79,1,7:41 PM,11:42,None,None,MISS Holiday 17' Pullup Jump Shot,None,None,5,201950,Jrue Holiday,1.610613e+09,Boston,Celtics,BOS,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,1
3,0022300563,8,4,0,1,7:41 PM,11:40,Porter REBOUND (Off:0 Def:1),None,None,None,None,4,1629007,Jontay Porter,1.610613e+09,Toronto,Raptors,TOR,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,1
4,0022300563,9,2,2,1,7:41 PM,11:35,MISS Barrett 26' 3PT Running Jump Shot,None,None,None,None,4,1629628,RJ Barrett,1.610613e+09,Toronto,Raptors,TOR,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,1
